In [3]:
using DelimitedFiles

In [85]:
function adjMatX(ct)
    # returns the adjacancy matrix corresponding to ct, with elements 1,2,3 corresponding to red,green and blue edges resepectively
    # ct: the coset talbel

    n=2*maximum(ct);
    adjMat = zeros(Int, n, n)
    for i in 1:(n÷2)
        adjMat[2*i, 2*i-1] = 1
        adjMat[2*i, 2*ct[6,i]-1] = 2
        adjMat[2*i, 2*ct[3,i]-1] = 3
    end
    adjMat += adjMat'
    return adjMat
end


function redFace(i, ct)
    # returns the red face next to vertex i, as an ordered list of vertices, going anti-clockwise
    # i: vertex index
    # ct: the coset talbel

    face = Int[]
    push!(face, 2i)
    push!(face, 2ct[6,i]-1)
    j = ct[4,ct[6,i]]
    while j != i
        push!(face, 2j)
        push!(face, 2ct[6,j]-1)
        j = ct[4,ct[6,j]]
    end
    return face
end

function greenFace(i, ct)
    # returns the green face next to vertex i, as an ordered list of vertices, going anti-clockwise
    # i: vertex index
    # ct: the coset talbel

    face = Int[]
    push!(face, 2i-1)
    push!(face, 2i)
    j = ct[3,i]
    while j != i
        push!(face, 2j-1)
        push!(face, 2j)
        j = ct[3,j]
    end
    return face
end

function blueFace(i, ct)
    # returns the blue face next to vertex i, as an ordered list of vertices, going anti-clockwise
    # i: vertex index
    # ct: the coset talbel

    face = Int[]
    push!(face, 2i)
    push!(face, 2i-1)
    j = ct[5,i]
    while j != i
        push!(face, 2j)
        push!(face, 2j-1)
        j = ct[5,j]
    end
    return face
end

function newFace(F, f)
    # check whether the face f is already in faces F
    # f: a face given by the list of the vertices around it
    # F: a list of faces
    for ff in F
        if length(findall(in(ff),f)) > 2
            return false
        end
    end
    return true
end


function facesHaveRightIntersection(F)
    # checks if the set of faces F has the correct intersection
    # F: the set of faces 

    for i in 1:length(F)
        for j in 1:(i-1)
            intersect_length = length(findall(in(F[i]),F[j]))
            if !(intersect_length == 0 || intersect_length == 2)
                return false
            end
        end
    end
    return true
end
function faces(ct)
    # return the list of faces, each face is represented by a list of vertices around the face, orderd as moving anticlockwise around the face
    # ct: the coset table

    n=2*maximum(ct)
    F = Vector{Vector{Int64}}()
    for i in 1:n÷2
        fs = [redFace(i, ct), greenFace(i, ct), blueFace(i, ct)]
        for f in fs
            if newFace(F, f)
                push!(F, f)
            end
        end
    end

    return F
    
end

function edges(ct)
    # return the list of edges, each edges is represented by a 2 element list holding the endpoint vertices
    # ct: the coset table
    Edges = Vector{Vector{Int}}()
    n=2*maximum(ct)
    for i in 1:(n÷2)
        push!(Edges, sort([2i, 2i-1]))
        push!(Edges, sort([2i, 2ct[6,i]-1]))
        push!(Edges, sort([2i, 2ct[3,i]-1]))
    end
    return Edges
end


function genus(ct)
    # computes the genus of the surface corresponding to ct
    # ct: the coset table

    V=2*maximum(ct)
    E=length(edges(ct))
    F=length(faces(ct))
    dg=(2 - V + E - F)
    if dg % 2 != 0
        println("ERROR in genus calculation: 2g is not even!")
        return -1
    end
    return dg÷2
end

function faceToEdges(f)
    # given an ordered list of vertices in a face f, returns the corresponding set of edges 
    # f: an ordered list of vertices around a face

    k=length(f)
    edges=[sort([f[i], f[i%k+1]]) for i in 1:k]
    return edges;
end

# F --d2--> E --d1--> V



# return the edges in the dual lattice, takes the boundary map from faces to edges as the argument
function dEdges(d2)
    d2T=transpose(d2)
    dE=[Vector{Int}(undef,2) for e in 1:size(d2T,2)]
    for e in 1:size(d2T,2)
        de=sort(findall(isone,d2T[:,e]))
        dE[e]=de
    end
    return dE 
    
end


"""
        geometry(ct)

return the edges E, the dual edges dE, the faces F, the boundary map 
from edges to vertices d1, the boundary map from faces to edges d2, the graph g and the dual graph dg

"""
function geometry(ct)


    n = 2*maximum(ct)

    F = faces(ct)

    E = edges(ct)

    d2_mat=_d2(E,F)

    d1_mat=_d1(E)
    
    g = _graph(E)

    dE = dEdges(d2_mat)
    dg = _graph(dE)

    return E,dE, F,d1_mat,d2_mat,g,dg
end



"""
    _graph(E)

return the graph g given the set of edges E. g[u] will be the vertices neighboring u.

# Arguments:
- 'E::Vector{Vector{Int}}' : Array of edges e, where each e=[u,v] represent an edge between vertex u and v. Assumes e=sort(e)
"""
function _graph(E)
    n=maximum(maximum.(E))
    g = [Vector{Int}(undef,0) for i=1:n]
    for e in E
        push!(g[e[1]],e[2])
        push!(g[e[2]],e[1])
    end
     return g
end




"""
    connectedComponent(g,v)
    
compute the connected subgraph of the graph g connected to vertext v
"""
function connectedComponent(g,v)
    visited=zeros(UInt8,length(g))
    _visitNeighbors!(g,v,visited)
    return findall(isone,visited)
end



"""
    _visitNeighbors!(g,v,visited)

visit vertex v (meaning update the corresponding element in visited to 1), and then visit its neighbors on the graph g.
it is a helper function to implement connectedComponent function  recursively 
"""
function _visitNeighbors!(g,v,visited)
    if visited[v]==1
        return 
    end

    visited[v]=1
    for u in g[v]
        _visitNeighbors!(g,u,visited)
    end
    return
end



"""
    removeEdge!(g,e)
    
return the graph g with the edge e=[u,v] removed.
does nothing if there is no edge between v and u in g.

"""
function removeEdge!(g,e)
    u,v=e
    setdiff!(g[v],[u])
    setdiff!(g[u],[v])
    return 
end


"""
    _d2(E,F)
    
return the d2 boundary operator from faces F to edges E.

# Arguments
- 'E::Vector{Vector{Int}}' : Array of edges e, where each e=[u,v] represent an edge between vertex u and v. Assumes e=sort(e)
- 'F::Vector{Vector{Int}}' : Array of faces f, where each f=[u,v,w,...] represent the vertices around face f, anticlockwise ordered.
"""
function _d2(E,F)
    d2_mat=zeros(Int, length(E), length(F))
    for j=1:length(F)
        f=F[j]
        for e in faceToEdges(f)
            i=findfirst(isequal(e),E)
            d2_mat[i,j]=1
        end
    end
    return d2_mat
end



"""
    _d1(E)
return the d1 boundary operator from edges E to the vertice.

# Arguments
- 'E::Vector{Vector{Int}}' : Array of edges e, where each e=[u,v] represent an edge between vertex u and v. Assumes e=sort(e)
"""
function _d1(E)

    n = maximum(maximum.(E))
    d1_mat=zeros(Int, n, length(E))
    for j=1:length(E)
        d1_mat[E[j][1],j]=1
        d1_mat[E[j][2],j]=1
    end

    return d1_mat
end

"""
    checkBoundaryOps(d1,d2)

test whether d1 and d2 are valid boundary operators. Basically check d1*d2=0 mod 2
"""
function checkBoundaryOps(d1,d2)
    if sum(mod.(d1*d2,2))>0
        println("ERORR: boundary of boundary is not empty")
        return false
    end
    return true
end


"""
    shortestPaths(graph, source)

    compute shortest paths to the source. returns 2 arrays, dist and prev. dist[u] is the shortest path from 
    source to u. prev[u] is the previous vertex on the shortest path from source to u, which can be used to 
    find the shortest path from the source to any vertex. 
"""
function shortestPaths(graph, source)
    n=length(graph)
    dist=[Inf for i=1:n]
    dist[source]=0
    prev=[-1 for i=1:n]
    queue=[i for i=1:n]
    while length(queue)>0
        u=argmin(u->dist[u],queue)
        for v in graph[u]
            if dist[u]+1 < dist[v]
                dist[v]=dist[u]+1
                prev[v]=u
            end
        end
        filter!(!isequal(u),queue)
    end

    return dist, prev
end




shortestPaths

In [84]:
ct=readdlm("cosetTable",Int)
E,dE,F,d1_mat,d2_mat,g,dg=geometry(ct);
# println(g[1])
# removeEdge!(g,[1,2])
# removeEdge!(g,[1,10])
# removeEdge!(g,[1,12])
# connectedComponent(g,2)
# length(F)

dist,prev=shortestPaths(g,1);
dist

24-element Vector{Float64}:
 0.0
 1.0
 4.0
 3.0
 4.0
 3.0
 2.0
 3.0
 2.0
 1.0
 ⋮
 3.0
 2.0
 3.0
 4.0
 3.0
 2.0
 3.0
 4.0
 3.0

In [45]:
dE

36-element Vector{Vector{Int64}}:
 [2, 3]
 [1, 3]
 [1, 2]
 [4, 5]
 [1, 5]
 [1, 4]
 [6, 7]
 [1, 7]
 [1, 6]
 [2, 7]
 ⋮
 [5, 6]
 [5, 9]
 [6, 9]
 [6, 10]
 [10, 12]
 [6, 12]
 [7, 11]
 [7, 12]
 [11, 12]

In [21]:
setdiff!(g[1],[10])

2-element Vector{Int64}:
  2
 12

In [22]:
g[1]

2-element Vector{Int64}:
  2
 12